In [1]:
import pyspark
import findspark
findspark.init()
sc = pyspark.SparkContext(appName="Pi")

In [1]:
# !export SPARK_HOME="/Users/ssv/spark-2.2.0-bin-hadoop2.7"
!echo $SPARK_HOME

/Users/ssv/spark-2.2.0-bin-hadoop2.7


In [2]:
import os
import sys

In [3]:
os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.11:1.5.0 pyspark-shell'
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.2.0
      /_/

Using Python version 3.6.2 (default, Sep 21 2017 18:29:43)
SparkSession available as 'spark'.


In [159]:
item_rdd = sc.textFile('/Users/ssv/newprolab/labs/lab06/ml-100k/u.item')

In [162]:
films = item_rdd.map(lambda x: (x.split('|')[0],  x.split('|')[1]))

In [5]:
data_rdd = sc.textFile('/Users/ssv/newprolab/labs/lab06/ml-100k/u.data')

## Lab06 begin

In [20]:
data_rdd.filter(lambda x: x.split('\t')[1] == '210').map(lambda x: (x.split('\t')[2], 1)) \
                                                        .reduceByKey(lambda x, y: x + y).sortByKey().take(5)

[('1', 3), ('2', 14), ('3', 79), ('4', 142), ('5', 93)]

In [21]:
data_rdd.map(lambda x: (x.split('\t')[2], 1)).reduceByKey(lambda x, y: x + y).sortByKey().take(5)

[('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)]

### Lab06sa

In [16]:
# Сумма рейтингов фильма
film_rat_sum = data_rdd.map(lambda x: (x.split('\t')[1], int(x.split('\t')[2]))).reduceByKey(lambda x, y: x + y)

In [307]:
# Количество человек n поставивших рейтинг фильму
film_rat_count = data_rdd.map(lambda x: (x.split('\t')[1], 1)).reduceByKey(lambda x, y: x + y)

In [140]:
top10_rates = film_rat_count.sortByKey().sortBy(lambda x: x[1], ascending=False)

In [ ]:
# Средний рейтинг фильма film_rat_sum/film_rat_count

In [151]:
mm = film_rat_sum.leftOuterJoin(film_rat_count)

In [153]:
mm.take(5)

[('346', (459, 126)),
 ('474', (825, 194)),
 ('265', (877, 227)),
 ('465', (303, 85)),
 ('451', (569, 170))]

In [40]:
mean_rating = film_rat_sum.leftOuterJoin(film_rat_count).map(lambda x: (x[0], x[1][0]/x[1][1]))

In [145]:
top10_average = mean_rating.sortByKey(ascending=True).sortBy(lambda x: x[1], ascending=False)

In [181]:
films.filter(lambda x: x[0] in fl).sortBy(keyfunc = lambda x: x[1], ascending=True).keys().take(10)

['1536', '1653', '814', '1201', '1189', '1467', '1500', '1599', '1293', '1122']

In [46]:
# Количество человек, оценивших фильм положительно rating >= 4
film_positiv_rat_count = data_rdd.filter(lambda x: int(x.split('\t')[2]) >= 4 ) \
                                        .map(lambda x: (x.split('\t')[1], 1)).reduceByKey(lambda x, y: x + y)

In [355]:
# Доля людей, оценивших фильм положительно film_positiv_rat_count/film_rat_count
mans_positiv_rat_count = film_rat_count.leftOuterJoin(film_positiv_rat_count) \
                                          .map(lambda x: (x[0], float(x[1][1])/float(x[1][0]) if x[1][1] != None else 0))

In [ ]:
# Глобальное среднее по всему датасету global_mean: 
сумма оценок по всем фильмам (ratings_summ) / количество всех оценок по всем фильмам (ratings_count)

In [53]:
ratings_count = data_rdd.map(lambda x: (x.split('\t')[2], 1)).count()

In [81]:
from operator import add
ratings_summ = data_rdd.map(lambda x: (int(x.split('\t')[2]))).reduce(add)

In [82]:
global_mean = ratings_summ/ratings_count

In [123]:
# Оценка поправленная на нехватку данных 
correct_mean_rating = film_rat_sum.leftOuterJoin(film_rat_count) \
                              .map(lambda x: (x[0], (x[1][0] + global_mean*10)/(x[1][1]+10)))

In [125]:
top10_rating = correct_mean_rating.sortByKey(ascending=True).sortBy(lambda x: x[1], ascending=False)

In [126]:
top10_rating.keys().take(10)

['318', '483', '64', '408', '169', '12', '603', '50', '114', '178']

In [415]:
import math
def wilson_score(x):
    n = float(x[1][0])
    p = float(x[1][1])
    z = float(2.576)
    result = 0
    
    if n != 0:
        ett = 1 / (1 + z*z/n)
        andra = p + z*z/(2*n)
        tredje = z * math.sqrt((p*(1-p)/n + (z*z/(4*n*n))))  
        result = ett * (andra - tredje)
    
    return (x[0], result)

In [410]:
wilson = film_rat_count.leftOuterJoin(mans_positiv_rat_count)

In [411]:
wilson = wilson.map(lambda x: (x[0], (x[1][0], x[1][1] if x[1][1] != None else 0)))

In [416]:
wilson = wilson.map(wilson_score)

In [419]:
wilson.sortBy(lambda x: x[1], ascending=False).keys().take(10)

['64', '318', '98', '479', '483', '50', '603', '427', '357', '12']

In [408]:
def wilson_score_(n, p):   
    z = 2.576
    result = 0
    
    if n != 0:
        ett = 1 / (1 + (1/n)*z*z)
        andra = p + (z*z/(2*n))
        tredje = z * math.sqrt( (1/n)*p*(1-p) + (1/(4*n*n))*z*z)  
        result = ett * (andra - tredje)
    
    print(n, p, ett, andra, tredje)
    print(z*z/(2*n), andra)
    return ( result)

In [ ]:
wilson_score_(85, 0.5529411764705883)

In [420]:
result = {
    'top10_rates'   : ['50', '258', '100', '181', '294', '286', '288', '1', '300', '121'],
    'top10_average' : ['1536', '1653', '814', '1201', '1189', '1467', '1500', '1599', '1293', '1122'],
    'top10_rating'  : ['318', '483', '64', '408', '169', '12', '603', '50', '114', '178'],
    'top10_lower'   : ['64', '318', '98', '479', '483', '50', '603', '427', '357', '12']
}

In [421]:
import json
with open('lab06s.json', 'w') as file:
    json.dump(result, file)

In [422]:
!scp -i ~/newprolab/keys/npl.pem ~/newprolab/spark/lektion01/lab06s.json sergey.sirosh@cluster.newprolab.com:/data/home/sergey.sirosh

lab06s.json                                   100%  350     7.5KB/s   00:00    


In [ ]:
#===============================================================================================================

In [80]:
import pandas as pd

In [85]:
df = pd.read_csv('/Users/ssv/newprolab/labs/lab06/ml-100k/u.data', sep='\t', header=None)

In [125]:
df[2][df[1] == 210].value_counts()

4    142
5     93
3     79
2     14
1      3
Name: 2, dtype: int64

In [123]:

result = { 'hist_film' : [0, 4, 28, 56, 44],
        'hist_all'  : [6110, 11370, 27145, 34174, 21201]  }

In [126]:
result = { 'hist_film' : [3, 14, 79, 142, 93],
           'hist_all'  : [6110, 11370, 27145, 34174, 21201]  }

####         result = { 'hist_film' : [0, 4, 28, 56, 44],
####                         'hist_all'  : [6110, 11370, 27145, 34174, 21201]  } hist_all - true